In [1]:
using PorousMaterials, Plots, JSON, DataFrames, CSV
plotly()

┌ Warning: ORCA.jl has been deprecated and all savefig functionality
│ has been implemented directly in PlotlyBase itself.
│ 
│ By implementing in PlotlyBase.jl, the savefig routines are automatically
│ available to PlotlyJS.jl also.
└ @ ORCA /home/sync/.julia/packages/ORCA/U5XaN/src/ORCA.jl:8


Plots.PlotlyBackend()

In [2]:
sbmof1 = Crystal("SBMOF-1.cif")
sbmof2 = Crystal("SBMOF-2.cssr")
forcefield = LJForceField("UFF", r_cutoff=12.8)
molecule = Molecule("Xe")

Molecule species: Xe
Center of mass (fractional coords): Cart([0.0; 0.0; 0.0])
Atoms:

	atom = Xe, x = [0.000, 0.000, 0.000]

In [3]:
df1 = CSV.File("data/sbmof1-xe-data.csv") |> 
     DataFrame |> 
     df -> select(
        df, 
        "P(mbar)" => (x -> x / 1000) => :pressure,
        "Weight(mg)" => (x -> x / 1000) => :mass_xe,
        "PercentMass" => (x -> x / 100 / 131.293 * 1000) => :loading
     )

,pressure,mass_xe,loading
,Float64,Float64,Float64
1,6.3e-5,0.0364861,3.04662e-7
2,0.000222,0.0364888,0.000597062
3,0.000398,0.03651,0.00506181
4,0.00058,0.0365438,0.0121557
5,0.000803,0.0365881,0.0214634
6,0.000985,0.0366281,0.0298717
7,0.004966,0.0374443,0.201338
8,0.009976,0.0382979,0.380808
9,0.01999,0.0394357,0.620555


In [4]:
sbmof1_pressures = df1.pressure   # bar
sbmof1_adsorptions = df1.loading;  # mmol/g

In [5]:
df2 = CSV.File("data/sbmof2-xe-data.csv") |> 
     DataFrame |> 
     df -> select(
        df,
        "P(torr)" => (x -> x / 750.062) => :pressure,
        "L(ccSTP/g)" => (x -> x / 22.4) => :loading
     )

,pressure,loading
,Float64,Float64
1,0.00406574,0.0423639
2,0.00609899,0.0637722
3,0.00812963,0.0850833
4,0.00915304,0.0957083
5,0.0101294,0.105825
6,0.0202933,0.210264
7,0.0300257,0.305586
8,0.0402655,0.402003
9,0.0492218,0.516198


In [6]:
sbmof2_pressures = df2.pressure   # bar
sbmof2_adsorptions = df2.loading;  # mmol/g

In [ ]:
sbmof1_results = stepwise_adsorption_isotherm(
    sbmof1,
    molecule,
    298.,
    sbmof1_pressures,
    forcefield,
    n_burn_cycles=5000,
    n_sample_cycles=5000,
);

In [ ]:
sbmof2_results = stepwise_adsorption_isotherm(
    sbmof2,
    molecule,
    298.,
    sbmof2_pressures,
    forcefield,
    n_burn_cycles=5000,
    n_sample_cycles=5000,
);

In [ ]:
sbmof1_sim_adsorptions = [r["⟨N⟩ (mmol/g)"] for r in sbmof1_results]
sbmof2_sim_adsorptions = [r["⟨N⟩ (mmol/g)"] for r in sbmof2_results];

In [7]:
sbmof1_sim_adsorptions = open("data/sbmof1-sim.json", "r") do io
    return [r["⟨N⟩ (mmol/g)"] for r in JSON.parse(io)]
end;

In [8]:
sbmof2_sim_adsorptions = open("data/sbmof2-sim.json", "r") do io
    return [r["⟨N⟩ (mmol/g)"] for r in JSON.parse(io)]
end;

In [9]:
p1 = plot(
    sbmof1_pressures,
    sbmof1_adsorptions,
    label="SBMOF-1 Data",
    xaxis=("Pressure (bar)", font(8)),
    yaxis=("Adsorption (mmol/g)", font(8)),
    legend=:bottomright,
)
plot!(p1, sbmof1_pressures, sbmof1_sim_adsorptions, label="SBMOF-1 Sim")

<!DOCTYPE html>
 
 
 Plots.jl

In [10]:
p2 = plot(
    sbmof2_pressures,
    sbmof2_adsorptions,
    label="SBMOF-2 Data",
    xaxis=("Pressure (bar)", font(8)),
    yaxis=("Adsorption (mmol/g)", font(8)),
    legend=:bottomright,
)
plot!(p2, sbmof2_pressures, sbmof2_sim_adsorptions, label="SBMOF-2 Sim")

<!DOCTYPE html>
 
 
 Plots.jl

In [ ]:
open("data/sbmof1-sim.json", "w") do io
    JSON.print(io, sbmof1_results, 4)
end

In [ ]:
open("data/sbmof2-sim.json", "w") do io
    JSON.print(io, sbmof2_results, 4)
end